# Trains and Tests PLDA

In [558]:
#!/usr/bin/env python3.7
import os
import sys
sys.path.insert(0, "/export/c10/lmorove1/PythonLibs/plda")
import plda
import numpy as np
import matplotlib.pyplot as plt
import kaldi_io
import argparse
import pickle
from sklearn.decomposition import PCA

import pandas as pd 

from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import mean_squared_error

#PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition

# GridSearchCV
from sklearn.model_selection import GridSearchCV

# Confusion matrix
from sklearn import metrics

In [425]:
DATA_PATH = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/'

!pwd && ls
os.chdir(DATA_PATH)
!pwd && ls

/export/c08/lmorove1/kaldi/egs/beatPDivec/v1
cmd.sh	 conf  exp    mfcc     pca_pldapy.ipynb  runFor.sh  run.sh~  steps
cmd.sh~  data  local  path.sh  README.txt	 run.sh     sid      utils
/export/c08/lmorove1/kaldi/egs/beatPDivec/v1
cmd.sh	 conf  exp    mfcc     pca_pldapy.ipynb  runFor.sh  run.sh~  steps
cmd.sh~  data  local  path.sh  README.txt	 run.sh     sid      utils


In [426]:
def pca(sFileTrai, sFileTest, iComponents):
# os.chdir("/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k")
# sFileTrai="/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k/exp/sre18nn/xvectors_Training_Fold1/xvector.scp"
# sOut = Folder where to store results 
# iComponents = nb of classes (nb of measurements_id? or nb of subject_ids?)
    dIvecTrai = { key:mat for key,mat in kaldi_io.read_vec_flt_scp(sFileTrai) }
    vTrai= pd.DataFrame((list(dIvecTrai.values())))
    vLTrai = np.array([x[-1] for x in np.array(list(dIvecTrai.keys()))])
    
    # FIXME : For realPD, we need more than -5 (CIS-PD subject_id is 4 characters long)
    # FIXME REAL-PD it's not only int 
    vTraiSubjectId = np.array(([int(x[-5:-1]) for x in np.array(list(dIvecTrai.keys()))]))

    dIvecTest = { key:mat for key,mat in kaldi_io.read_vec_flt_scp(sFileTest) }
    vTest=np.array(list(dIvecTest.values()), dtype=float)
    vLTest=np.array([x[-1] for x in np.array(list(dIvecTest.keys()))])
    vTestSubjectId = np.array([int(x[-5:-1]) for x in np.array(list(dIvecTest.keys()))])

    print('before transform : ', vTrai.shape)
    print('before transform : ', vTest.shape)

    if isinstance(iComponents, str):
            iComponents=int(iComponents)


    pca = PCA(n_components=iComponents, svd_solver='randomized', whiten=True)
    pca.fit(vTrai)


    vTraiPCA=pca.transform(vTrai)
    vTestPCA=pca.transform(vTest)

    print('after transform : ', vTraiPCA.shape)
    print('after transform : ', vTestPCA.shape)
    return vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId

In [427]:
sFileTrai = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/ivectors_Training_Fold0/ivector.scp'
sFileTest = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/ivectors_Testing_Fold0/ivector.scp'
iComponents = 50 


In [553]:
vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId = pca(sFileTrai, sFileTest, iComponents)


before transform :  (1403, 300)
before transform :  (364, 300)
after transform :  (1403, 50)
after transform :  (364, 50)


In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.datasets import load_iris
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline

# # To compute the final score as per the challenge
# mse_training_per_subjectid=[]
# mse_test_per_subjectid=[]
# train_nb_files_per_subjectid=[]
# test_nb_files_per_subjectid=[]

# # To compute the mean accuracy
# glob_trai_pred=[]
# glob_test_pred=[]
# glob_trai_true=[]
# glob_test_true=[]
    
# for subject_id in np.unique(vTraiSubjectId):
#     # GridSearchCV
# #     parameters = {'n_neighbors':[1,3,6,10,15]}

#     k_range = list(range(1, 15))
# #     param_grid = dict(n_neighbors=k_range)
# #     print(param_grid)

#     # instantiate the grid
# #     grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')

#     # fit the grid with data
# #     grid.fit(vTraiPCA_subjectid, vLTrai_subjectid)

#     # view the complete results (list of named tuples)
# #     print(grid.cv_results_['mean_test_score'])

#     print('----- ' + str(subject_id) + '----- ')
#     knn = KNeighborsClassifier(n_neighbors=6)
    
#     # Filter vTraiPCA and vLTraiPCA for one subject_id
#     indices_subject_id = np.where(vTraiSubjectId == subject_id) # HAPPY
#     vTraiPCA_subjectid = vTraiPCA[indices_subject_id]
#     vLTrai_subjectid = vLTrai[indices_subject_id]
    
#     # Filter vTestPCA and vLTestPCA for one subject_id
#     indices_subject_id = np.where(vTestSubjectId == subject_id)
#     vTestPCA_subjectid = vTestPCA[indices_subject_id]
#     vLTest_subjectid = vLTest[indices_subject_id]

#     knn.fit(vTraiPCA_subjectid, vLTrai_subjectid)
    
# #     y_labels = knn.predict(vTraiPCA_subjectid)
# #     display(y_labels)

#     print('Training accuracy: ', knn.score(vTraiPCA_subjectid, vLTrai_subjectid))
#     print('Testing accuracy: ', knn.score(vTestPCA_subjectid, vLTest_subjectid))
    
#     # Predicting on the training and test data 
#     predictionsTrai = knn.predict(vTraiPCA_subjectid)
#     predictions = knn.predict(vTestPCA_subjectid)
    
#     # Converting all strings to int for MSE 
#     vLTrai_subjectid = [int(i) for i in vLTrai_subjectid]
#     predictionsTrai = [int(i) for i in predictionsTrai]
#     vLTest_subjectid = [int(i) for i in vLTest_subjectid]
#     predictions = [int(i) for i in predictions]
 
#     # Computing the accuracy 
#     glob_trai_pred=np.append(glob_trai_pred,predictionsTrai,axis=0)
#     glob_test_pred=np.append(glob_test_pred,predictions,axis=0)
#     glob_trai_true=np.append(glob_trai_true,vLTrai_subjectid,axis=0)
#     glob_test_true=np.append(glob_test_true,vLTest_subjectid,axis=0)
    
#     a = [a - b for a, b in zip(vLTrai_subjectid, predictionsTrai)]
#     print('abs substract : ', np.abs(a))
    
#     # Building a list of the MSEk 
#     mse_training_per_subjectid = np.append(mse_training_per_subjectid,
#                                            (mean_squared_error(vLTrai_subjectid, predictionsTrai) /  len(vLTrai_subjectid)))
#     mse_test_per_subjectid = np.append(mse_test_per_subjectid,
#                                         (mean_squared_error(vLTest_subjectid, predictions) / len(vLTest_subjectid)))
#     train_nb_files_per_subjectid.append(len(vLTrai_subjectid))
#     test_nb_files_per_subjectid.append(len(vLTest_subjectid))

# #     do_confusion_matrix(vLTest_subjectid, predictions)

# print('Global training accuracy: {}'.format((glob_trai_true == glob_trai_pred).mean()))
# print('Global testing accuracy: {}'.format((glob_test_true == glob_test_pred).mean()))

In [574]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# To compute the final score as per the challenge
mse_training_per_subjectid=[]
mse_test_per_subjectid=[]
train_nb_files_per_subjectid=[]
test_nb_files_per_subjectid=[]

# To compute the mean accuracy
glob_trai_pred=[]
glob_test_pred=[]
glob_trai_true=[]
glob_test_true=[]
    
for subject_id in np.unique(vTraiSubjectId):
    # GridSearchCV
#     parameters = {'n_neighbors':[1,3,6,10,15]}
    knn2 = KNeighborsClassifier()
    k_range = list(range(1, 15))
    param_grid = dict(n_neighbors=k_range)
#     print(param_grid)

    # instantiate the grid
    grid = GridSearchCV(knn2, param_grid, cv=5, scoring='accuracy')

    # fit the grid with data
    grid.fit(vTraiPCA_subjectid, vLTrai_subjectid)

    #check top performing n_neighbors value
    print(grid.best_params_)
    
    print(grid.cv_results_)
    #check mean score for the top performing value of n_neighbors
    print(grid.best_score_)

    print('----- ' + str(subject_id) + '----- ')
    knn = KNeighborsClassifier(n_neighbors=7)
    
    # Filter vTraiPCA and vLTraiPCA for one subject_id
    indices_subject_id = np.where(vTraiSubjectId == subject_id) # HAPPY
    vTraiPCA_subjectid = vTraiPCA[indices_subject_id]
    vLTrai_subjectid = vLTrai[indices_subject_id]
    
    # Filter vTestPCA and vLTestPCA for one subject_id
    indices_subject_id = np.where(vTestSubjectId == subject_id)
    vTestPCA_subjectid = vTestPCA[indices_subject_id]
    vLTest_subjectid = vLTest[indices_subject_id]

    knn.fit(vTraiPCA_subjectid, vLTrai_subjectid)
    
#     y_labels = knn.predict(vTraiPCA_subjectid)
#     display(y_labels)

    print('Training accuracy: ', knn.score(vTraiPCA_subjectid, vLTrai_subjectid))
    print('Testing accuracy: ', knn.score(vTestPCA_subjectid, vLTest_subjectid))
    
    # Predicting on the training and test data 
    predictionsTrai = knn.predict(vTraiPCA_subjectid)
    predictions = knn.predict(vTestPCA_subjectid)
    
    # Converting all strings to int for MSE 
    vLTrai_subjectid = [int(i) for i in vLTrai_subjectid]
    predictionsTrai = [int(i) for i in predictionsTrai]
    vLTest_subjectid = [int(i) for i in vLTest_subjectid]
    predictions = [int(i) for i in predictions]
 
    # Computing the accuracy 
    glob_trai_pred=np.append(glob_trai_pred,predictionsTrai,axis=0)
    glob_test_pred=np.append(glob_test_pred,predictions,axis=0)
    glob_trai_true=np.append(glob_trai_true,vLTrai_subjectid,axis=0)
    glob_test_true=np.append(glob_test_true,vLTest_subjectid,axis=0)
    
    a = [a - b for a, b in zip(vLTrai_subjectid, predictionsTrai)]
    print('abs substract : ', np.abs(a))
    
    # Building a list of the MSEk 
    mse_training_per_subjectid = np.append(mse_training_per_subjectid,
                                           (mean_squared_error(vLTrai_subjectid, predictionsTrai) /  len(vLTrai_subjectid)))
    mse_test_per_subjectid = np.append(mse_test_per_subjectid,
                                        (mean_squared_error(vLTest_subjectid, predictions) / len(vLTest_subjectid)))
    train_nb_files_per_subjectid.append(len(vLTrai_subjectid))
    test_nb_files_per_subjectid.append(len(vLTest_subjectid))

#     do_confusion_matrix(vLTest_subjectid, predictions)

print('Global training accuracy: {}'.format((glob_trai_true == glob_trai_pred).mean()))
print('Global testing accuracy: {}'.format((glob_test_true == glob_test_pred).mean()))

/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


{'n_neighbors': 7}
{'mean_fit_time': array([0.00070157, 0.00066128, 0.00045357, 0.00070777, 0.00067053,
       0.00044765, 0.00045943, 0.00047684, 0.00046668, 0.0004715 ,
       0.00048494, 0.0004529 , 0.00044117, 0.00045714]), 'rank_test_score': array([14, 11, 12, 12, 10,  9,  1,  1,  1,  1,  1,  1,  1,  1],
      dtype=int32), 'std_score_time': array([6.31231548e-05, 2.44299969e-04, 3.03829472e-05, 2.84995481e-05,
       2.04719026e-04, 4.51302125e-05, 2.07322743e-05, 1.06243599e-04,
       8.09508848e-05, 3.73699809e-05, 5.19747253e-05, 3.05365712e-05,
       3.81922453e-05, 2.09943771e-05]), 'split0_test_score': array([0.59375, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875,
       0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875, 0.71875]), 'split4_train_score': array([1.   , 0.784, 0.736, 0.72 , 0.72 , 0.72 , 0.736, 0.736, 0.736,
       0.736, 0.736, 0.736, 0.736, 0.736]), 'std_train_score': array([0.        , 0.01225666, 0.01344117, 0.01150384, 0.01059487,
       0.009

/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'n_neighbors': 2}
{'mean_fit_time': array([0.00105305, 0.00084176, 0.00070534, 0.00067663, 0.00062327,
       0.00055275, 0.00056686, 0.00053244, 0.00048747, 0.00045385,
       0.00051823, 0.00043473, 0.00039887, 0.00039372]), 'rank_test_score': array([ 3,  1,  5,  5,  9,  5,  9, 11, 13, 13,  5, 11,  1,  3],
      dtype=int32), 'std_score_time': array([1.43369527e-04, 1.45996652e-04, 8.20676785e-05, 1.14756289e-04,
       8.73732239e-05, 2.27273816e-05, 9.82949347e-05, 6.11324206e-05,
       2.36317917e-05, 4.33023277e-05, 7.41865145e-05, 2.96374913e-05,
       1.87745510e-05, 2.98375875e-05]), 'split0_test_score': array([0.1875, 0.25  , 0.0625, 0.1875, 0.125 , 0.25  , 0.25  , 0.125 ,
       0.125 , 0.125 , 0.125 , 0.0625, 0.0625, 0.0625]), 'split4_train_score': array([1.        , 0.55555556, 0.44444444, 0.5       , 0.31481481,
       0.38888889, 0.38888889, 0.35185185, 0.40740741, 0.38888889,
       0.37037037, 0.38888889, 0.33333333, 0.35185185]), 'std_train_score': array([0.       

/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


...

{'n_neighbors': 11}
{'mean_fit_time': array([0.00040164, 0.00039754, 0.00039845, 0.00041628, 0.00039902,
       0.0004014 , 0.00040245, 0.00039663, 0.00039897, 0.00039821,
       0.00040088, 0.0004003 , 0.00040193, 0.00040154]), 'rank_test_score': array([14,  9, 13,  8, 11,  9, 11,  6,  6,  2,  1,  2,  4,  4],
      dtype=int32), 'std_score_time': array([2.33348340e-05, 1.92836861e-05, 1.71617661e-05, 2.52616851e-04,
       1.61871994e-05, 3.11128116e-05, 1.74826997e-05, 2.64412116e-05,
       1.94796004e-05, 2.07632881e-05, 3.70360375e-05, 1.81934467e-05,
       1.77754762e-05, 2.92859548e-05]), 'split0_test_score': array([0.28571429, 0.57142857, 0.6       , 0.54285714, 0.48571429,
       0.54285714, 0.54285714, 0.65714286, 0.62857143, 0.62857143,
       0.65714286, 0.65714286, 0.62857143, 0.65714286]), 'split4_train_score': array([1.        , 0.79104478, 0.73134328, 0.73134328, 0.7238806 ,
       0.71641791, 0.68656716, 0.68656716, 0.69402985, 0.67910448,
       0.67164179, 0.6492537

/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


{'n_neighbors': 5}
{'mean_fit_time': array([0.00046368, 0.00033674, 0.00032501, 0.00032849, 0.00032701,
       0.00032587, 0.00033078, 0.00033073, 0.00033479, 0.00030489,
       0.00025449, 0.00025349, 0.000247  , 0.00024858]), 'rank_test_score': array([ 6, 10, 12,  5,  1,  1,  3,  3,  6, 12,  6, 10, 12,  6],
      dtype=int32), 'std_score_time': array([2.53345633e-04, 5.68403093e-05, 1.20608699e-05, 6.57793360e-06,
       4.81204064e-06, 3.76369233e-06, 6.35929885e-06, 1.05088246e-05,
       2.42073652e-05, 2.59640350e-05, 2.57060785e-05, 1.11986707e-05,
       5.63515816e-06, 2.23495768e-05]), 'split0_test_score': array([0.42857143, 0.42857143, 0.28571429, 0.42857143, 0.57142857,
       0.57142857, 0.42857143, 0.42857143, 0.42857143, 0.28571429,
       0.42857143, 0.28571429, 0.        , 0.14285714]), 'split4_train_score': array([1.        , 0.43478261, 0.43478261, 0.39130435, 0.43478261,
       0.56521739, 0.39130435, 0.43478261, 0.39130435, 0.39130435,
       0.30434783, 0.34782609

{'n_neighbors': 14}
{'mean_fit_time': array([0.00038762, 0.00034943, 0.00035305, 0.00038018, 0.00035982,
       0.00036097, 0.00034542, 0.00033975, 0.00034599, 0.00034575,
       0.00036402, 0.00036173, 0.00035796, 0.00040412]), 'rank_test_score': array([13, 14, 10, 12,  6,  6, 11,  2,  6,  4,  4,  6,  2,  1],
      dtype=int32), 'std_score_time': array([3.43637190e-05, 1.55205917e-05, 1.36420436e-05, 2.27821866e-04,
       2.09943771e-05, 1.13473218e-05, 1.01724019e-05, 1.34317503e-05,
       3.43322754e-06, 5.70333920e-06, 1.56912032e-05, 1.31354574e-05,
       1.01884826e-05, 2.20164551e-05]), 'split0_test_score': array([0.4375, 0.4375, 0.4375, 0.625 , 0.75  , 0.625 , 0.5625, 0.5625,
       0.5625, 0.5625, 0.5   , 0.5625, 0.5   , 0.5   ]), 'split4_train_score': array([1.        , 0.7       , 0.66666667, 0.6       , 0.61666667,
       0.6       , 0.56666667, 0.58333333, 0.58333333, 0.56666667,
       0.56666667, 0.6       , 0.61666667, 0.61666667]), 'std_train_score': array([0.      

/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [575]:
from sklearn.metrics import mean_squared_error
from math import sqrt 

def final_score(mse_per_subjectid, nb_files_per_subject_id):
    numerator = np.sum([a * b for a, b in zip(np.sqrt(nb_files_per_subject_id), mse_per_subjectid)])
    denominator = np.sum(np.sqrt(nb_files_per_subject_id))
    print('numerator : ', numerator)
    print('denominator : ', denominator)
    print('Final score : ', np.divide(numerator, denominator))

In [556]:
final_score(mse_training_per_subjectid, train_nb_files_per_subjectid)
final_score(mse_test_per_subjectid, test_nb_files_per_subjectid)

numerator :  2.9956883253194277
denominator :  138.07717596772696
Final score :  0.021695753149091168
numerator :  6.065415875730853
denominator :  70.2868377625761
Final score :  0.08629518795851071


In [543]:
def do_confusion_matrix(y_test, predictions):
    print('y test : ', np.unique(y_test))
    LABELS_NEW = np.unique(y_test)
    n_classes=np.unique(y_test)
    print("Confusion Matrix:")
    confusion_matrix = metrics.confusion_matrix(y_test, predictions)
    print(confusion_matrix)
    normalised_confusion_matrix = np.array(confusion_matrix, dtype=np.float32)/np.sum(confusion_matrix)*100

    print("")
    print("Confusion matrix (normalised to % of total test data):")
    print(normalised_confusion_matrix)

    # Plot Results:
    width = 12
    height = 12
    plt.figure(figsize=(width, height))
    plt.imshow(
        normalised_confusion_matrix,
        interpolation='nearest',
        cmap=plt.cm.rainbow
    )
    plt.title("Confusion matrix \n(normalised to % of total test data)")
    plt.colorbar()
    tick_marks = np.arange(n_classes)
    plt.xticks(tick_marks, LABELS_NEW, rotation=90)
    plt.yticks(tick_marks, LABELS_NEW)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()